In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## SUMMARY
For Week 20, we would like you to build a map.

Our data this week comes from InsideAirbnb and was provided by Frederic Frey for the VizIt Syndey community conference. “Inside Airbnb is a mission driven project that provides data and advocacy about Airbnb's impact on residential communities. They work towards a vision where communities are empowered with data and information to understand, decide and control the role of renting residential homes to tourists.” Please reference their site as your data source.



Our data set contains the following fields:

- id - Airbnb's unique identifier for the listing
- name - Name of the Listing
- host_id - Airbnb’s unique identifier for the host/user
- host_name - Name of the host (usually just the first name)
- neighbourhood
- latitude
- longitude
- room_type - entire place, private room, shared room
- price - daily price in local currency
- minimum_nights - minimum number of night stay fr the listing
- number_of_reviews - number of reviews the listing has
- reviews_per_month - average number of review per month
- calculated_host_listings_count - the number of listings the host has in the current scrape
- number_of_reviews_ltm - number of review the listing has in the last 12 months

In [7]:
# Prettymaps
import prettymaps
# Vsketch
import vsketch
# OSMNX
import osmnx as ox
# Matplotlib-related
import matplotlib.font_manager as fm
from matplotlib import pyplot as plt
from descartes import PolygonPatch
# Shapely
from shapely.geometry import *
from shapely.affinity import *
from shapely.ops import unary_union

# EDA - Exploratory Data Analysis

In [12]:
df = pd.read_csv('../Data/Sydney Airbnb Listings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24022 entries, 0 to 24021
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              24022 non-null  float64
 1   name                            24022 non-null  object 
 2   host_id                         24022 non-null  int64  
 3   host_name                       24020 non-null  object 
 4   neighbourhood                   24022 non-null  object 
 5   latitude                        24022 non-null  float64
 6   longitude                       24022 non-null  float64
 7   room_type                       24022 non-null  object 
 8   price                           24022 non-null  int64  
 9   minimum_nights                  24022 non-null  int64  
 10  number_of_reviews               24022 non-null  int64  
 11  reviews_per_month               18028 non-null  float64
 12  calculated_host_listings_count  